In [1]:
#source https://github.com/HimariO/FairFace/blob/master/inference.py

In [2]:
#!pip install cmake
#!pip install dlib
#!pip install loguru

In [3]:
#!conda install -c conda-forge dlib

In [4]:
# Два первых способа установки не родошли, только этот:
#sudo apt-get install cmake
#wget https://files.pythonhosted.org/packages/05/57/e8a8caa3c89a27f80bc78da39c423e2553f482a3705adc619176a3a24b36/dlib-19.17.0.tar.gz
#tar -xvzf dlib-19.17.0.tar.gz
#cd dlib-19.17.0/
#sudo python3 setup.py install

In [5]:
import os
import glob
import json
import fire
import random
from multiprocessing import Pool
from collections import Counter

import dlib
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import datasets, models, transforms
from tqdm.notebook import tqdm

#На gpu у меня опять не та версия cuda
device = torch.device("cpu")#torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
repo_dir = '/media/alex/Storage/coding/HimariO/HatefulMemesChallenge'
gqa_box_anno = os.path.join(repo_dir, 'data/hateful_memes/box_annos.json')
meme_img_dir = os.path.join(repo_dir, 'data/hateful_memes/img_clean')
output_path = os.path.join(repo_dir, 'data/hateful_memes/face_race_boxes.json')
(gqa_box_anno, meme_img_dir, output_path )

('/media/alex/Storage/coding/HimariO/HatefulMemesChallenge/data/hateful_memes/box_annos.json',
 '/media/alex/Storage/coding/HimariO/HatefulMemesChallenge/data/hateful_memes/img_clean',
 '/media/alex/Storage/coding/HimariO/HatefulMemesChallenge/data/hateful_memes/face_race_boxes.json')

In [7]:

def draw_boxes(image, boxes, class_names, scores, max_boxes=10, min_score=0.3):
    """Overlay labeled boxes on an image with formatted scores and label names."""
    _colors = list(ImageColor.colormap.values())
    colors = []
    for c in _colors:
        if type(c) is str:
            colors.append(c + 'AA')
    # colors = [c + 'AA' for c in colors]

    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                                16)
    except IOError:
        print("Font not found, using default font.")
        font = ImageFont.load_default()

    for i in range(min(boxes.shape[0], max_boxes)):
        if scores[i] >= min_score:
            xmin, ymin, xmax, ymax = tuple(boxes[i])
            display_str = "{}: {}%".format(class_names[i].decode("ascii"),
                                            int(100 * scores[i]))
            color = colors[hash(class_names[i]) % len(colors)]
            image_pil = Image.fromarray(np.uint8(image)).convert("RGB")
            draw_bounding_box_on_image(
                image_pil,
                ymin,
                xmin,
                ymax,
                xmax,
                color,
                font,
                display_str_list=[display_str])
            np.copyto(image, np.array(image_pil))
    return image

In [8]:
def load_dlib_model():
    cnn_face_detector = dlib.cnn_face_detection_model_v1('dlib_models/mmod_human_face_detector.dat')
    sp = dlib.shape_predictor('dlib_models/shape_predictor_5_face_landmarks.dat')

    model_fair_7 = torchvision.models.resnet34(pretrained=True)
    model_fair_7.fc = torch.nn.Linear(model_fair_7.fc.in_features, 18)
    model_fair_7.load_state_dict(
        torch.load('face_recognition_fair_face_models/fair_face_models/res34_fair_align_multi_7_20190809.pt', map_location=torch.device('cpu'))
    )
    model_fair_7 = model_fair_7.to(device)
    model_fair_7.eval()

    return cnn_face_detector, sp, model_fair_7


In [9]:
def box_coverage(box_a, box_b):
    assert (box_a[2] > 1 and box_b[2] > 1) or (box_a[2] <= 1 and box_b[2] <= 1)
    area_a = (box_a[2] - box_a[0]) * (box_a[3] - box_a[1])
    area_b = (box_b[2] - box_b[0]) * (box_b[3] - box_b[1])
    
    larger = box_a if area_a > area_b else box_b
    small = box_b if area_a > area_b else box_a

    w = small[2] - small[0]
    h = small[3] - small[1]
    ocr_l_to_img_r = max(min(larger[2] - small[0], w), 0)
    ocr_r_to_img_l = max(min(small[2] - larger[0], w), 0)
    cover_w = min(ocr_l_to_img_r, ocr_r_to_img_l)
    
    ocr_t_to_img_b = max(min(larger[3] - small[1], h), 0)
    ocr_b_to_img_t = max(min(small[3] - larger[1], h), 0)
    cover_h = min(ocr_t_to_img_b, ocr_b_to_img_t)
    return (cover_h * cover_w) / (w * h)

In [10]:
def converage_nms(primary_set, sec_set, indies=False, drop=True, threshold=0.4):
    keep_sec = []
    keep = []
    for j, s_box in enumerate(sec_set):
        covers = [0]
        for i, p_box in enumerate(primary_set):
            cov = box_coverage(p_box, s_box)
            covers.append(cov)
        
        if drop:
            if max(covers) < threshold:
                keep_sec.append(s_box)
                keep.append(j)
        else:
            if max(covers) >= threshold:
                keep_sec.append(s_box)
                keep.append(j)
    if indies:
        return keep_sec, keep
    else:
        return keep_sec

In [11]:
def predidct_age_gender_race(images, model_fair_7):
    race_7 = [
        'White',
        'Black',
        'Latino_Hispanic',
        'East Asian',
        'Southeast Asian',
        'Indian',
        'Middle Eastern',
    ]
    gender = [
        'Male',
        'Female'
    ]
    
    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    # img pth of face images
    face_names = []
    # list within a list. Each sublist contains scores for all races. Take max for predicted race
    race_scores_fair = []
    gender_scores_fair = []
    age_scores_fair = []
    
    race_preds_fair = []
    gender_preds_fair = []
    age_preds_fair = []
    
    race_scores_fair_4 = []
    race_preds_fair_4 = []

    predicts = []
    for index, image in enumerate(images):
        image = trans(image)
        image = image.view(1, 3, 224, 224)  # reshape image to match model dimensions (1 batch size)
        image = image.to(device)

        # fair
        outputs = model_fair_7(image)
        outputs = outputs.cpu().detach().numpy()
        outputs = np.squeeze(outputs)

        race_outputs = outputs[:7]
        gender_outputs = outputs[7:9]
        age_outputs = outputs[9:18]

        race_score = np.exp(race_outputs) / np.sum(np.exp(race_outputs))
        gender_score = np.exp(gender_outputs) / np.sum(np.exp(gender_outputs))
        age_score = np.exp(age_outputs) / np.sum(np.exp(age_outputs))

        race_pred = race_7[np.argmax(race_score)]
        gender_pred = gender[np.argmax(gender_score)]
        age_pred = np.argmax(age_score)

        race_scores_fair.append(race_score)
        gender_scores_fair.append(gender_score)
        age_scores_fair.append(age_score)

        race_preds_fair.append(race_pred)
        gender_preds_fair.append(gender_pred)
        age_preds_fair.append(age_pred)
        
        print(race_pred, gender_pred)
        print(race_score, gender_score)
        predicts.append((race_pred, gender_pred))
    return predicts

In [12]:
def predict(anno, meme_img_dir, cnn_face_detector, sp, model_fair_7, debug=False):
    img_name = anno['img_name']
    img_path = os.path.join(meme_img_dir, img_name)
    img = dlib.load_rgb_image(img_path)
    h, w = img.shape[:2]
    
    face_crops, dlib_boxes = dlib_detect(img, cnn_face_detector, sp)
    
    for bs in anno['boxes_and_score']:
        if 'box' not in bs:
            # mul_w = w if bs['xmax'] < 1 else 1.0
            # mul_h = h if bs['ymax'] < 1 else 1.0
            mul_w = w
            mul_h = h
            bs['box'] = [
                bs['xmin'] * mul_w,
                bs['ymin'] * mul_h,
                bs['xmax'] * mul_w,
                bs['ymax'] * mul_h
            ]
    
    face_box = [
        [int(b) for b in bs['box'][:4]]
        for bs in anno['boxes_and_score']
        if bs['class_name'].lower() in ['face', 'human face']
    ]
    face_box = converage_nms(dlib_boxes, face_box)
    
    head_box = [
        [int(b) for b in bs['box'][:4]]
        for bs in anno['boxes_and_score']
        if bs['class_name'] in ['head', 'human head']
    ]
    head_box = converage_nms(dlib_boxes + face_box, head_box)
    
    hair_box = [
        [int(b) for b in refine_hair_box(bs['box'][:4], h, w)]
        for bs in anno['boxes_and_score']
        if bs['class_name'] in ['hair', 'human hair']
    ]
    hair_box = converage_nms(dlib_boxes + face_box + head_box, hair_box)
    gqa_det_crops = [
        img[b[1]: b[3], b[0]: b[2], ...]
        for b in face_box + head_box
    ]

    if debug:
        _boxes = np.array(dlib_boxes + face_box + head_box)
        _img = img.copy()
        draw_boxes(
            _img,
            _boxes,
            [f"{i}".encode() for i in range(len(_boxes))],
            [1.0] * len(_boxes),
        )
        plt.figure(figsize=(15, 15))
        plt.imshow(_img)
        plt.title(f"{i}")
        plt.show()

    all_crop_boxes = dlib_boxes + face_box + head_box
    all_cls_pred = predidct_age_gender_race(face_crops + gqa_det_crops, model_fair_7)
    assert len(all_crop_boxes) == len(all_cls_pred)
    return all_crop_boxes, all_cls_pred


In [13]:
def dlib_detect(img, cnn_face_detector, sp, 
                default_max_size=800, size=300, padding=0.25):
    # plt.imshow(img)
    # plt.show()

    old_height, old_width, _ = img.shape

    if old_width > old_height:
        new_width, new_height = default_max_size, int(default_max_size * old_height / old_width)
    else:
        new_width, new_height =  int(default_max_size * old_width / old_height), default_max_size
#     img = dlib.resize_image(img, rows=new_height, cols=new_width)

    dets = cnn_face_detector(img, 1)
    num_faces = len(dets)

    print(f"Find {num_faces} faces")
#         continue
    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()
    face_boxes = []
    for detection in dets:
        rect = detection.rect
        print(rect.height(), rect.tl_corner(), rect.br_corner())
        face_boxes.append([
            rect.tl_corner().x,
            rect.tl_corner().y,
            rect.br_corner().x,
            rect.br_corner().y,
        ])
        faces.append(sp(img, rect))
    if len(dets) > 0:
        images = dlib.get_face_chips(img, faces, size=size, padding = padding)
    else:
        images = []
    
    # for face_img in images:
    #     plt.imshow(face_img)
    #     plt.show()
    return images, face_boxes


In [14]:
def _detect_race(gqa_boxes, meme_img_dir):
    cnn_face_detector, sp, model_fair_7 = load_dlib_model()
    results = {}
    for i, anno in enumerate(gqa_boxes):
        crop_boxes, cls_pred = predict(anno, meme_img_dir, cnn_face_detector, sp, model_fair_7)
        results[anno['img_name']] = {
            'face_boxes': crop_boxes,
            'face_race': [c[0] for c in cls_pred],
            'face_gender': [c[1] for c in cls_pred],
        }
    return results

In [15]:
def detect_gqa_race_mp(gqa_box_anno, meme_img_dir, output_path, debug=False, worker=4):
    with open(gqa_box_anno, 'r') as f:
        gqa_boxes = json.load(f)
        random.shuffle(gqa_boxes)
    
    gather_result = {}
    
    for box in tqdm(gqa_boxes):
        r = _detect_race([box], meme_img_dir)
        gather_result.update(r)
    
    with open(output_path, 'w') as f:
        json.dump(gather_result, f)
  

In [16]:
detect_gqa_race_mp(gqa_box_anno, meme_img_dir, output_path)

Find 1 faces
48 (289, 41) (336, 88)
White Male
[9.2709219e-01 4.5887842e-05 3.3261667e-03 2.1252694e-02 3.0470295e-03
 1.6653018e-04 4.5069553e-02] [9.9974781e-01 2.5217774e-04]
Find 0 faces
Middle Eastern Male
[0.16315348 0.09357902 0.17743632 0.03671307 0.03827828 0.03414774
 0.45669204] [0.92913944 0.07086058]
East Asian Male
[0.02507685 0.00472971 0.04744866 0.7992641  0.0805032  0.00591958
 0.03705788] [0.9230482 0.0769518]
Find 1 faces
353 (248, 30) (600, 382)
White Female
[9.8744375e-01 1.8893533e-06 5.4446066e-04 6.5121138e-05 3.6616249e-05
 1.0457081e-05 1.1897691e-02] [0.01095169 0.9890483 ]
Find 1 faces
294 (87, 188) (381, 481)
White Male
[9.8261386e-01 2.3547183e-05 1.0673327e-03 2.0864831e-05 3.1422322e-05
 2.7368579e-04 1.5969260e-02] [9.9974889e-01 2.5110645e-04]
Find 1 faces
246 (131, 85) (376, 330)
White Female
[9.9965042e-01 6.4930964e-07 2.9498115e-05 3.7095066e-07 3.9262653e-07
 2.3593730e-06 3.1621757e-04] [0.03954061 0.9604594 ]
Find 1 faces
171 (198, 162) (368, 3

Find 3 faces
41 (181, 305) (220, 345)
41 (177, 633) (216, 673)
41 (177, 113) (216, 153)
Latino_Hispanic Male
[0.00328347 0.13584551 0.38750318 0.05989    0.35424957 0.05729761
 0.00193063] [9.9998862e-01 1.1346632e-05]
Black Male
[8.9997787e-04 6.7947936e-01 2.8201407e-03 8.1275675e-06 2.3867062e-04
 3.1610975e-01 4.4395076e-04] [9.9999833e-01 1.6416500e-06]
Black Male
[2.4742694e-04 8.1677538e-01 8.5695100e-04 1.3231607e-06 4.7499401e-05
 1.8186030e-01 2.1110901e-04] [9.999995e-01 4.881667e-07]
Find 1 faces
294 (165, 218) (459, 511)
White Male
[0.34637418 0.00970154 0.02648331 0.30847543 0.25037003 0.01378865
 0.04480692] [0.61863816 0.3813619 ]
Find 0 faces
White Male
[7.7729970e-01 5.8744143e-05 1.1746179e-02 9.1075867e-02 3.8177595e-03
 5.2098738e-04 1.1548084e-01] [9.9982560e-01 1.7437154e-04]
Find 0 faces
Indian Male
[2.4316267e-05 1.2724864e-02 2.8738277e-03 3.5501907e-06 2.0847692e-04
 9.8412919e-01 3.5798861e-05] [0.87616926 0.12383078]
Find 1 faces
143 (200, 70) (342, 212)
La

Find 1 faces
69 (237, 78) (306, 146)
White Female
[9.87562537e-01 6.35111473e-06 8.01502354e-03 9.25060885e-06
 1.13429505e-05 3.42882304e-05 4.36113682e-03] [4.7330435e-05 9.9995267e-01]
Find 1 faces
143 (242, 79) (383, 221)
East Asian Male
[7.7621557e-04 4.6187539e-05 2.2060186e-02 5.2849793e-01 4.4593248e-01
 2.6134816e-03 7.3489624e-05] [9.9988949e-01 1.1050622e-04]
Find 1 faces
171 (215, 104) (385, 274)
Black Female
[3.2908953e-04 9.3282855e-01 1.7173938e-02 9.7015378e-04 2.3721160e-02
 2.4841474e-02 1.3571145e-04] [0.44835132 0.5516486 ]
Find 0 faces
White Female
[9.7753268e-01 1.2386349e-04 7.3523377e-03 8.5481917e-03 1.7915344e-03
 2.3777058e-04 4.4136364e-03] [0.15381831 0.8461817 ]
Find 2 faces
295 (117, 106) (410, 400)
294 (414, 136) (708, 429)
White Male
[9.8301232e-01 6.4719920e-06 9.6878586e-03 2.3390383e-05 5.0325707e-05
 1.5026006e-03 5.7170633e-03] [9.999654e-01 3.457962e-05]
White Male
[0.44069928 0.00590746 0.3196897  0.00277364 0.00814173 0.16803028
 0.05475787] [0.

Find 0 faces
Find 0 faces
Find 0 faces
Find 1 faces
70 (210, 57) (278, 126)
White Female
[9.9449956e-01 5.0759336e-06 1.7898855e-03 1.2220157e-04 6.6845962e-05
 1.0940208e-04 3.4071226e-03] [8.6445577e-04 9.9913555e-01]
Find 0 faces
Find 6 faces
83 (393, 107) (475, 189)
69 (175, 110) (243, 178)
69 (728, 172) (796, 240)
83 (617, 182) (699, 264)
83 (28, 82) (110, 164)
69 (514, 131) (582, 199)
Black Female
[3.5824653e-02 7.2170550e-01 2.1270737e-01 1.9089598e-04 5.4619729e-04
 2.3385439e-02 5.6399782e-03] [8.0467125e-06 9.9999195e-01]
White Female
[9.9998343e-01 2.2960844e-08 7.7635341e-06 2.6032366e-08 6.1936809e-08
 1.5695947e-07 8.5063093e-06] [2.2903312e-06 9.9999768e-01]
Latino_Hispanic Female
[1.88482866e-01 5.83717701e-05 6.75100863e-01 4.28076349e-02
 5.82169779e-02 2.01913398e-02 1.51419565e-02] [1.5063449e-05 9.9998498e-01]
White Female
[7.2145075e-01 3.1679083e-05 1.0632700e-01 1.8076715e-04 4.2242705e-04
 5.2806488e-03 1.6630685e-01] [3.2583432e-06 9.9999672e-01]
White Female


White Female
[0.40820932 0.00248695 0.18208686 0.00433802 0.00155968 0.05980513
 0.3415141 ] [0.47451103 0.5254889 ]
Middle Eastern Male
[0.32453743 0.00399552 0.07255859 0.00873697 0.00236479 0.03491957
 0.5528871 ] [0.6130774  0.38692254]
East Asian Female
[0.06847735 0.00206253 0.10241276 0.4072682  0.3981385  0.00864372
 0.01299701] [0.01125779 0.9887422 ]
Middle Eastern Male
[2.42355719e-01 3.01111098e-02 1.32688120e-01 1.33424226e-04
 2.28267265e-04 1.16765145e-02 5.82806885e-01] [0.9898229  0.01017707]
Find 1 faces
245 (116, 135) (360, 379)
Black Male
[8.1841991e-04 9.9687475e-01 9.9373364e-04 1.1192001e-04 9.8308874e-04
 9.4688177e-05 1.2332824e-04] [0.9941404  0.00585961]
Find 0 faces
Find 3 faces
83 (53, 158) (135, 240)
82 (186, 84) (268, 165)
49 (443, 153) (490, 201)
Middle Eastern Male
[2.0917784e-01 4.3038098e-04 1.4240342e-02 3.7841030e-06 7.4599125e-06
 4.3354426e-03 7.7180475e-01] [9.9999768e-01 2.2970862e-06]
White Female
[9.9835408e-01 5.9698635e-07 1.0902234e-03 3.47

...

Find 2 faces
99 (351, 358) (450, 456)
143 (199, 127) (340, 269)
White Male
[0.824794   0.00289873 0.01775345 0.11823212 0.01516917 0.00101663
 0.02013586] [0.65919954 0.3408005 ]
Middle Eastern Male
[0.20432945 0.00339039 0.04431427 0.00400534 0.00738998 0.0370512
 0.6995194 ] [9.9972981e-01 2.7016204e-04]
Find 0 faces
Find 0 faces
Find 1 faces
119 (276, 199) (394, 317)
Black Female
[4.2594259e-04 9.0029281e-01 2.4000706e-02 1.4124892e-04 4.1375025e-03
 7.0822261e-02 1.7952878e-04] [1.12669564e-04 9.99887347e-01]
East Asian Male
[0.3107677  0.01830262 0.12400616 0.34461844 0.08623531 0.03190598
 0.08416368] [0.90778506 0.09221487]
Black Male
[0.11469589 0.64137226 0.03004588 0.13171536 0.07202125 0.00682062
 0.00332879] [0.8524312  0.14756879]
East Asian Male
[0.06282719 0.05668271 0.05460415 0.544532   0.2475246  0.01781487
 0.01601437] [0.5356695  0.46433046]
Find 0 faces
Find 1 faces
58 (192, 616) (249, 673)
White Male
[7.3531288e-01 3.5509478e-02 4.6841234e-02 7.2501029e-04 6.58691

Find 2 faces
99 (91, 87) (190, 185)
41 (1, 153) (40, 193)
White Male
[9.7567940e-01 7.1481511e-04 1.7666815e-02 2.2959162e-04 1.2192007e-04
 1.0609721e-03 4.5264857e-03] [9.9975723e-01 2.4279466e-04]
White Female
[9.8603225e-01 2.0971528e-04 4.1133813e-03 9.6669770e-04 6.7914574e-04
 1.1741798e-03 6.8246466e-03] [0.04714812 0.95285183]
Find 2 faces
41 (185, 153) (224, 193)
82 (61, 57) (143, 138)
White Male
[0.58051556 0.151256   0.0480514  0.00140843 0.00214188 0.02085943
 0.19576737] [0.956426   0.04357395]
White Male
[8.8404399e-01 2.9562509e-03 1.5095205e-02 8.0053054e-04 3.4737418e-04
 1.5468451e-03 9.5209867e-02] [9.9975127e-01 2.4870652e-04]
Find 2 faces
205 (220, 478) (424, 682)
246 (199, 60) (444, 305)
White Male
[4.9247065e-01 3.2576697e-03 3.3187713e-02 2.6103322e-05 1.6017734e-04
 2.3846798e-02 4.4705087e-01] [9.9993026e-01 6.9745060e-05]
Middle Eastern Male
[3.3663964e-01 1.2127615e-03 5.7675101e-02 1.7835147e-04 1.1021884e-03
 2.1272359e-02 5.8191961e-01] [9.9976426e-01 2.

Find 1 faces
58 (105, 46) (162, 103)
Southeast Asian Female
[0.00750617 0.00114947 0.02698222 0.34703735 0.61173165 0.00496323
 0.00062987] [6.435700e-05 9.999356e-01]
Find 1 faces
119 (198, 133) (316, 251)
White Female
[7.6280338e-01 1.3972791e-02 1.4366408e-01 4.3382116e-02 1.6821040e-02
 3.9856305e-04 1.8958049e-02] [0.35192236 0.64807767]
Find 1 faces
58 (100, 42) (156, 99)
White Female
[6.8759948e-01 1.3408888e-04 7.1827933e-02 3.2777043e-05 2.5425024e-05
 2.0046884e-03 2.3837563e-01] [4.1149233e-05 9.9995881e-01]
Find 4 faces
41 (189, 73) (228, 113)
41 (81, 77) (120, 117)
99 (181, 236) (279, 334)
48 (121, 67) (168, 114)
White Female
[9.8211259e-01 2.4677015e-06 9.9802797e-04 2.2693419e-05 1.0285482e-05
 1.1895520e-04 1.6734987e-02] [4.0854917e-05 9.9995917e-01]
White Female
[9.0502870e-01 2.6729971e-05 2.0075854e-02 1.2159199e-03 1.3593351e-03
 8.3517155e-04 7.1458250e-02] [5.9388884e-05 9.9994063e-01]
Latino_Hispanic Female
[0.32923254 0.01331571 0.4833591  0.00059527 0.00129105

White Male
[0.6978915  0.0034989  0.00731734 0.02021548 0.00846306 0.00340173
 0.25921202] [0.9352128  0.06478723]
White Male
[0.81033117 0.00350549 0.01769154 0.049044   0.01973409 0.00691072
 0.09278296] [0.5901473  0.40985265]
White Female
[0.45444378 0.01578724 0.25029704 0.05122608 0.01163992 0.02365721
 0.1929487 ] [0.30073488 0.6992652 ]
Find 1 faces
58 (215, 69) (272, 126)
Black Male
[0.00582799 0.6559649  0.15672567 0.00254347 0.01814028 0.15661484
 0.00418289] [9.999826e-01 1.747584e-05]
Find 2 faces
119 (262, 478) (380, 596)
83 (293, 50) (375, 132)
White Male
[9.6972841e-01 7.4349307e-05 1.8891541e-03 7.2058574e-03 3.6042123e-03
 1.3321842e-04 1.7364752e-02] [9.9928164e-01 7.1831490e-04]
White Male
[0.771299   0.00080063 0.00464443 0.12738009 0.09391864 0.00111543
 0.00084187] [0.99864805 0.00135199]
Find 0 faces
Find 2 faces
83 (476, 191) (558, 273)
99 (279, 149) (377, 247)
Black Female
[7.7325160e-05 9.7868216e-01 3.6721125e-03 6.8210975e-05 8.6080434e-04
 1.6623905e-02 1.

Find 1 faces
142 (270, 90) (412, 231)
Black Female
[9.65200888e-05 8.35219502e-01 7.58485217e-03 3.75049021e-05
 1.35614285e-02 1.42824218e-01 6.75916497e-04] [0.00102693 0.998973  ]
Find 1 faces
171 (97, 248) (267, 418)
White Male
[9.9503136e-01 2.4280074e-05 3.2371469e-03 3.1254246e-04 2.4280867e-04
 7.0135487e-05 1.0816013e-03] [9.99985933e-01 1.41233095e-05]
Find 1 faces
69 (224, 85) (292, 153)
White Female
[0.4583729  0.00995315 0.28539085 0.00291979 0.00440807 0.02079246
 0.21816269] [5.1967945e-04 9.9948037e-01]
Find 1 faces
49 (116, 28) (164, 76)
White Male
[8.9961427e-01 2.3332101e-05 4.4229846e-03 2.8308114e-02 2.9684491e-03
 1.8698181e-04 6.4475812e-02] [9.999125e-01 8.748568e-05]
Find 0 faces
East Asian Male
[0.04766869 0.01248375 0.0406386  0.68924665 0.14058203 0.04937627
 0.02000403] [0.65204495 0.34795502]
White Male
[0.7655175  0.01878169 0.04631428 0.00802397 0.00412926 0.01718801
 0.14004526] [0.98862284 0.01137717]
Find 1 faces
83 (443, 53) (525, 135)
White Male
[9.

In [ ]:
#Пример результата
"34721.png": {"face_boxes": [[289, 41, 336, 88]], "face_race": ["White"], "face_gender": ["Male"]},
"47629.png": {"face_boxes": [[417, 57, 489, 141], [189, 65, 296, 153]], "face_race": ["Middle Eastern", "East Asian"], "face_gender": ["Male", "Male"]},

In [17]:
with open(gqa_box_anno, 'r') as f:
    gqa_boxes = json.load(f)
print(gqa_boxes[0])
_detect_race([gqa_boxes[0]], meme_img_dir)

{'img_name': '01235.png', 'boxes_and_score': [{'ymin': 0.2707633674144745, 'xmin': 0.39449542760849, 'ymax': 0.618645429611206, 'xmax': 0.5975348353385925, 'score': 0.9788097739219666, 'class_name': 'Human face', 'class_id': 502}, {'ymin': 0.1195806935429573, 'xmin': 0.050002746284008026, 'ymax': 0.9922588467597961, 'xmax': 0.9625630378723145, 'score': 0.9190647006034851, 'class_name': 'Man', 'class_id': 308}, {'ymin': 0.49269676208496094, 'xmin': 0.07834305614233017, 'ymax': 0.9805120229721069, 'xmax': 0.9698421359062195, 'score': 0.7085372805595398, 'class_name': 'Clothing', 'class_id': 433}, {'ymin': 0.45603710412979126, 'xmin': 0.020553115755319595, 'ymax': 0.9691803455352783, 'xmax': 0.3579131066799164, 'score': 0.21738770604133606, 'class_name': 'Human arm', 'class_id': 503}]}
Find 1 faces
119 (210, 79) (328, 197)
Middle Eastern Male
[4.1799635e-02 1.0545440e-03 9.6204944e-02 2.5493049e-05 1.0217457e-04
 4.1344479e-01 4.4736835e-01] [9.9996126e-01 3.8761355e-05]


{'01235.png': {'face_boxes': [[210, 79, 328, 197]],
  'face_race': ['Middle Eastern'],
  'face_gender': ['Male']}}

In [ ]:
#Результат сохраняется в face_race_boxes.json